In [9]:
from flygym.examples.common import get_data_path
import flygym.util as util
import numpy as np

#xml parser
from lxml import etree

In [10]:
seqikpy_skel_corresp = {
    "RFCoxa":"RF_Coxa", "RFFemur":"RF_Femur", "RFTibia":"RF_Tibia", "RFTarsus1":"RF_Tarsus", "RFTarsus5":"RF_Claw",
    "LFCoxa":"LF_Coxa", "LFFemur":"LF_Femur", "LFTibia":"LF_Tibia", "LFTarsus1":"LF_Tarsus", "LFTarsus5":"LF_Claw",
    "RMCoxa":"RM_Coxa", "RMFemur":"RM_Femur", "RMTibia":"RM_Tibia", "RMTarsus1":"RM_Tarsus", "RMTarsus5":"RM_Claw",
    "LMCoxa":"LM_Coxa", "LMFemur":"LM_Femur", "LMTibia":"LM_Tibia", "LMTarsus1":"LM_Tarsus", "LMTarsus5":"LM_Claw",
    "RHCoxa":"RH_Coxa", "RHFemur":"RH_Femur", "RHTibia":"RH_Tibia", "RHTarsus1":"RH_Tarsus", "RHTarsus5":"RH_Claw",
    "LHCoxa":"LH_Coxa", "LHFemur":"LH_Femur", "LHTibia":"LH_Tibia", "LHTarsus1":"LH_Tarsus", "LHTarsus5":"LH_Claw",
    "RPedicel":"R_Antenna_base", "RArista":"R_Antenna_edge", "LPedicel":"L_Antenna_base", "LArista":"L_Antenna_edge",
    "Head":"Neck", "Thorax":"Thorax_mid", "Abdomen":"Abdomen_mid", "RWing":"R_wing", "LWing":"L_wing"
    }

In [16]:
all_skeletons = {}

for xml_variant in ["deepfly3d", "deepfly3d_old", "deepfly3d_original", "seqik", "seqik_old"]:
    mujoco_config = util.load_config() 
    xml_path = get_data_path("flygym", "data") / mujoco_config["paths"]["mjcf"][xml_variant]

    # load the xml at xml_path and read line by line
    tree = etree.parse(xml_path)
    root = tree.getroot()

    # read line by line
    skeleton = {}
    for i, line in enumerate(tree.iter()):
        if line.tag == "body":
            parent = line.getparent()
            parent_name = parent.get("name")
            body_pos = np.array(line.get("pos").strip().split(" ")).astype(float)
            if not parent is None and parent_name in skeleton:
                curr_name = line.get("name")
                # find joint if exists
                joint = line.findall("joint")
                if len(joint) > 0 and False:
                    joint_pos = np.array(joint[0].get("pos").strip().split(" ")).astype(float)
                    if not np.all(joint_pos == 0.0):
                        print(f"Warning: {curr_name} in {xml_variant} has non-zero joint pos")
                else:
                    joint_pos = np.zeros(3)
                    #print(f"Warning: {curr_name} in {xml_variant} has no joint")

                skeleton[curr_name] = skeleton[parent_name] + body_pos + joint_pos
            else:
                skeleton[line.get("name")] = body_pos

    seqikpy_skeleton = {v: skeleton[k] for k, v in seqikpy_skel_corresp.items() if k in skeleton}
    all_skeletons[xml_variant] = seqikpy_skeleton

In [17]:
# save all of them in a python file with xml_variant = all_skeletons[xml_variant]
from pathlib import Path
out_path = Path("/Users/stimpfli/Desktop/sequential-inverse-kinematics/seqikpy/flygym_skeletons.py")
out_path = Path("/Users/stimpfli/Desktop/nmf2-paper/grooming/data/flygym_skeletons.py")

with open(out_path, "w") as f:
    f.write("import numpy as np\n\n")
    for xml_variant, skeleton in all_skeletons.items():
        f.write(xml_variant.upper() + " = " + "{" + "\n")
        for k, v in skeleton.items():
            f.write(f'\t"{k}": np.array({v.tolist()}),\n')
        f.write("}\n\n")
